# Neo4j Python driver: examples


## Requirement


### Installation

    pip install neo4j

### Full documentation


Full documentation is available at: https://neo4j.com/docs/api/python-driver/current/


## Simple usage: session

In [1]:
from neo4j import GraphDatabase

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "<YOUR_PASSWORD>"))
driver

In [3]:
session = driver.session()
session

In [4]:
cypher = "MATCH (n:Node) RETURN n"
result = session.run(cypher)
result

In [5]:
# fetching a single record:
record = result.peek()
record

<Record n=<Node id=49 labels=frozenset({'Node'}) properties={'name': 'A'}>>

In [6]:
node = record.get("n")
node

<Node id=49 labels=frozenset({'Node'}) properties={'name': 'A'}>

In [7]:
node.get("name")

'A'

In [8]:
dict(node.items())

{'name': 'A'}

In [9]:
# getting all results:
for record in result:
    print(record.get("n").get("name"))

A
B
C
D
BB
AA
Z


In [10]:
session.close()

The last operation can be avoided by wrapping the session into a context manager using the `with` statement, like one would do to open a file:

```
with open("my_file.csv", "r") as f:
    data = f.readLines()
  
# here the file is automatically closed
```

In [11]:
with driver.session() as session:
    for record in session.run(cypher):
        print(record.get("n").get("name"))

A
B
C
D
BB
AA
Z


In [12]:
# if you try to run any Cypher now, it will fail because the sesison is closed when leaving the `with` block:
# session.run(cypher)

## Transactions

In [13]:
with driver.session() as session:
    # start a new transaction
    tx = session.begin_transaction()
    # run cypher...
    # tx.run(statement_1)
    # tx.run(statement_2)
    # push changes to the graph
    tx.commit()

### Successful transaction

In [14]:
with driver.session() as session:
    # start a new transaction
    tx = session.begin_transaction()
    # run cypher...
    tx.run("CREATE (n:Node {name: 'BB'})")
    tx.run("CREATE (n:Node {name: 'AA'})")
    # push changes to the graph
    tx.commit()

### Unsuccessful transaction

In [15]:
with driver.session() as session:
    # start a new transaction
    tx = session.begin_transaction()
    # run cypher...
    tx.run("CREATE (n:Node {name: 'CC'})")
    tx.run("CREATE (n:Node name: 'DD')")  # this query is not valid Cypher!
    # push changes to the graph
    tx.commit()

CypherSyntaxError: Invalid input 'n': expected whitespace, comment, NodeLabel, MapLiteral, a parameter, a parameter (old syntax), ')' or a relationship pattern (line 1, column 16 (offset: 15))
"CREATE (n:Node name: 'DD')"
                ^

In [16]:
# we can check that there is no Node with name 'CC'
with driver.session() as session:
    print(session.run("MATCH (n:Node {name: 'CC'}) RETURN n").data())

[]


## Queries with parameters

In [17]:
create_node = """
CREATE (n:Node {name: $name})
RETURN id(n) as id
"""

update_node = """
MATCH (n:Node)
WHERE id(n) = $id
SET n.value = $value
RETURN n
"""

In [18]:
with driver.session() as session:
    # start a new transaction
    tx = session.begin_transaction()
    # run cypher...
    r1 = tx.run(create_node, {"name": "Z"})
    node_id = r1.single().get("id")
    r2 = tx.run(update_node, {"id": node_id, "value": 10})
    result = r2.single().get("n")
    print(dict(result.items()))
    # push changes to the graph
    tx.commit()

{'name': 'Z', 'value': 10}


## Another interesing function

`.data()` method for BoltStatementResult and Record returns the data as a list of dict:

In [19]:
cypher

'MATCH (n:Node) RETURN n'

In [20]:
with driver.session() as session:
    # run cypher...
    result = session.run(cypher)
    
    display(result.data())

[{'n': {'name': 'A'}},
 {'n': {'name': 'B'}},
 {'n': {'name': 'C'}},
 {'n': {'name': 'D'}},
 {'n': {'name': 'BB'}},
 {'n': {'name': 'AA'}},
 {'n': {'name': 'BB'}},
 {'n': {'name': 'Z', 'value': 10}},
 {'n': {'name': 'AA'}},
 {'n': {'name': 'Z', 'value': 10}}]